# Importando bibliotecas

In [31]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,recall_score,roc_curve,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import plotly.graph_objects as go
import keras
import joblib
import pickle

# Importe os dados e modelos

In [4]:
dados_tratados = pickle.load(open('dados/dados_tratados.pickle', 'rb'))
x_treino = dados_tratados['x_treino']
y_treino = dados_tratados['y_treino']
x_teste = dados_tratados['x_teste']
y_teste = dados_tratados['y_teste']

print(x_treino.shape, y_treino.shape, x_teste.shape, y_teste.shape) 

(59606, 20) (59606,) (15891, 20) (15891,)


In [5]:
## Cat Boost
cat_boost = CatBoostClassifier()
cat_boost.load_model("modelos_salvos/cat_boost_model.cbm") 

## Random Forest
#random_forest = joblib.load('modelos_salvos/random_forest_model.pkl')

# Avaliando os modelos

In [9]:
random_pred = cat_boost.predict(x_teste)
random_pred_prob = cat_boost.predict_proba(x_teste)

In [10]:
print(classification_report(y_teste.values, random_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     15518
           1       0.85      0.68      0.75       373

    accuracy                           0.99     15891
   macro avg       0.92      0.84      0.87     15891
weighted avg       0.99      0.99      0.99     15891



In [12]:
confusion_matrix(y_teste, random_pred)

array([[15473,    45],
       [  121,   252]])

In [23]:
y_proba = cat_boost.predict_proba(x_teste)[:,1]

In [25]:
cat_boost.predict_proba(x_teste)

array([[9.99946622e-01, 5.33777494e-05],
       [9.99976381e-01, 2.36190322e-05],
       [9.99984505e-01, 1.54948977e-05],
       ...,
       [9.99999464e-01, 5.35624962e-07],
       [9.99752082e-01, 2.47917917e-04],
       [9.99998897e-01, 1.10323407e-06]])

In [24]:
y_proba

array([5.33777494e-05, 2.36190322e-05, 1.54948977e-05, ...,
       5.35624962e-07, 2.47917917e-04, 1.10323407e-06])

In [37]:
import matplotlib.pyplot as plt
lista_valor_monetario = []
lista_range = []
for i in range(0, 10):
    y_proba = cat_boost.predict_proba(x_teste)[:,1]
    y_pred_redes_threshold = (y_proba > i/10).astype(int)
    confusio = confusion_matrix(y_teste, y_pred_redes_threshold)
    lista_valor_monetario.append((confusio[0][0] * 0)+(confusio[1][1] * 25)+(confusio[0][1]*10)+(confusio[1][0]*500))
    lista_range.append(i/10)
fig = go.Figure()
fig.add_trace(go.Scatter(x=lista_range, y=lista_valor_monetario, mode='lines', name='Predição do Custo de manutenção por nivel de threshold'))
fig.update_layout(
        title="Predição do Custo de manutenção por nivel de threshold",
        xaxis_title="Threshold",
        yaxis_title="Custo de Manutenção",
        legend_title="Legenda",
        height=700,
        width=1000,
        yaxis=dict(range=[0, max(lista_valor_monetario) * 1.1]),
        )
#plt.plot(lista_range, lista_valor_monetario)

In [43]:
y_proba = cat_boost.predict_proba(x_teste)[:,1]
y_pred_redes_threshold = (y_proba > 0.2).astype(int)
confusion_matrix(y_teste, y_pred_redes_threshold)

array([[15456,    62],
       [   91,   282]])

dados_tratados = pickle.load(open('dados/dados_tratados.pickle', 'rb'))
x_treino = dados_tratados['x_treino']
y_treino = dados_tratados['y_treino']
x_teste = dados_tratados['x_teste']
y_teste = dados_tratados['y_teste']

print(x_treino.shape, y_treino.shape, x_teste.shape, y_teste.shape) 